In [253]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import random
import seaborn as sn
import datetime as dt
import re
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.utils import resample
from nltk.corpus import stopwords
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem import PorterStemmer
from sklearn.preprocessing import LabelEncoder
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from gensim.models import Word2Vec, KeyedVectors
from itertools import combinations
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, r2_score, mean_squared_error, SCORERS, f1_score, classification_report, confusion_matrix, roc_auc_score, roc_curve, precision_recall_curve
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.linear_model import Lasso, LassoLars, LinearRegression, Ridge
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.svm import SVC
from xgboost import XGBClassifier, XGBRegressor, XGBRFRegressor
%matplotlib inline

In [12]:
first_camp = pd.read_csv('/Users/skylark/Desktop/healthcare/First_Health_Camp_Attended.csv')
second_camp = pd.read_csv('/Users/skylark/Desktop/healthcare/Second_Health_Camp_Attended.csv')
third_camp = pd.read_csv('/Users/skylark/Desktop/healthcare/Third_Health_Camp_Attended.csv')
health_camp_details = pd.read_csv('/Users/skylark/Desktop/healthcare/Health_Camp_Detail.csv')
patient_profile = pd.read_csv('/Users/skylark/Desktop/healthcare/Patient_Profile.csv')
train = pd.read_csv('/Users/skylark/Desktop/healthcare/Train.csv')
test = pd.read_csv('/Users/skylark/Desktop/healthcare/test_l0Auv8Q.csv')
data_dictionary = pd.read_excel('/Users/skylark/Desktop/healthcare/Data_Dictionary.xlsx')
sample = pd.read_csv('/Users/skylark/Desktop/healthcare/sample_submmission.csv')

In [13]:
first_camp.drop('Unnamed: 4', axis=1, inplace=True)

In [14]:
first_camp

,Patient_ID,Health_Camp_ID,Donation,Health_Score
0,506181,6560,40,0.439024
1,494977,6560,20,0.097561
2,518680,6560,10,0.048780
3,509916,6560,30,0.634146
4,488006,6560,20,0.024390
...,...,...,...,...
6213,502728,6575,10,0.509804
6214,511088,6575,20,0.078431
6215,507608,6575,40,0.627451
6216,488046,6575,50,0.686275


In [16]:
train = train.merge(first_camp, how='left', left_on=['Patient_ID','Health_Camp_ID'], right_on=['Patient_ID','Health_Camp_ID'])

In [17]:
train.drop('Donation', axis=1, inplace=True)

In [20]:
train = train.merge(second_camp, how='left', left_on=['Patient_ID','Health_Camp_ID'], right_on=['Patient_ID','Health_Camp_ID'])

In [23]:
train = train.merge(third_camp, how='left', left_on=['Patient_ID','Health_Camp_ID'], right_on=['Patient_ID','Health_Camp_ID'])

In [25]:
train[train.Number_of_stall_visited==0]

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5,Health_Score,Health Score,Number_of_stall_visited,Last_Stall_Visited_Number
1322,514108,6527,02-May-05,0,0,0,0,0,NaN,NaN,0.0,0.0
1411,486530,6527,08-May-05,12,0,0,0,4,NaN,NaN,0.0,0.0
2131,497040,6527,15-Jun-05,0,0,0,0,0,NaN,NaN,0.0,0.0
10570,489505,6527,15-Jun-05,0,0,0,0,0,NaN,NaN,0.0,0.0
11838,515663,6527,01-May-05,0,0,0,0,0,NaN,NaN,0.0,0.0
12269,516582,6527,20-May-05,0,0,0,0,0,NaN,NaN,0.0,0.0
12974,496394,6527,15-Jun-05,0,0,0,0,0,NaN,NaN,0.0,0.0
15558,492766,6527,06-May-05,0,0,0,0,0,NaN,NaN,0.0,0.0
15995,525007,6527,15-Jun-05,0,0,0,0,0,NaN,NaN,0.0,0.0
16143,506873,6527,15-Jun-05,0,0,0,0,0,NaN,NaN,0.0,0.0


In [27]:
train['Outcome'] = np.where((train.Health_Score.isnull()==False) | (train['Health Score'].isnull()==False) | (train.Number_of_stall_visited.isnull()==False), 1, 0)

In [31]:
train.loc[train.Number_of_stall_visited==0, 'Outcome'] = 0

In [41]:
train = train[train.Registration_Date.notnull()]

In [45]:
train = train.merge(patient_profile, how='left', left_on='Patient_ID', right_on='Patient_ID')
train = train.merge(health_camp_details, how='left', left_on='Health_Camp_ID', right_on='Health_Camp_ID')

In [47]:
test = test.merge(patient_profile, how='left', left_on='Patient_ID', right_on='Patient_ID')
test = test.merge(health_camp_details, how='left', left_on='Health_Camp_ID', right_on='Health_Camp_ID')

In [50]:
train = train[['Patient_ID', 'Health_Camp_ID', 'Registration_Date', 'Var1', 'Var2',
       'Var3', 'Var4', 'Var5', 'Health_Score', 'Health Score',
       'Number_of_stall_visited', 'Last_Stall_Visited_Number',
       'Online_Follower', 'LinkedIn_Shared', 'Twitter_Shared',
       'Facebook_Shared', 'Income', 'Education_Score', 'Age',
       'First_Interaction', 'City_Type', 'Employer_Category',
       'Camp_Start_Date', 'Camp_End_Date', 'Category1', 'Category2',
       'Category3', 'Outcome']]

In [52]:
data = pd.concat([train, test])

In [58]:
data = data.drop('Last_Stall_Visited_Number', axis=1)

In [67]:
data['Income'] = data.Income.replace('None', 7)

In [69]:
data['Income'] = pd.to_numeric(data['Income'])

In [78]:
data['Education_Score'] = pd.to_numeric(data.Education_Score.replace('None', np.nan), errors='ignore')

In [90]:
data.Education_Score.fillna(0, inplace=True)

In [93]:
data['Age'] = data['Age'].replace('None', np.nan)

In [95]:
data['Age'] = pd.to_numeric(data['Age'], errors='ignore')

In [104]:
data.Age.fillna(30, inplace=True)

In [106]:
data.City_Type.fillna('J', inplace=True)

In [108]:
data.Employer_Category.fillna('Others', inplace=True)

In [110]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 110193 entries, 0 to 35248
Data columns (total 24 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Patient_ID         110193 non-null  int64  
 1   Health_Camp_ID     110193 non-null  int64  
 2   Registration_Date  110193 non-null  object 
 3   Var1               110193 non-null  int64  
 4   Var2               110193 non-null  int64  
 5   Var3               110193 non-null  int64  
 6   Var4               110193 non-null  int64  
 7   Var5               110193 non-null  int64  
 8   Online_Follower    110193 non-null  int64  
 9   LinkedIn_Shared    110193 non-null  int64  
 10  Twitter_Shared     110193 non-null  int64  
 11  Facebook_Shared    110193 non-null  int64  
 12  Income             110193 non-null  int64  
 13  Education_Score    110193 non-null  float64
 14  Age                110193 non-null  float64
 15  First_Interaction  110193 non-null  object 
 16  Cit

In [159]:
dates = ['Registration_Date', 'Camp_Start_Date', 'Camp_End_Date', 'First_Interaction']
for date in dates:
    data[date] = pd.to_datetime(data[date])
    
Vars = ['Var1', 'Var2', 'Var3', 'Var4', 'Var5']
for var in Vars:
    data[var] = pd.to_numeric(data[var])

In [115]:
data.isnull().sum()

Patient_ID               0
Health_Camp_ID           0
Registration_Date        0
Var1                     0
Var2                     0
Var3                     0
Var4                     0
Var5                     0
Online_Follower          0
LinkedIn_Shared          0
Twitter_Shared           0
Facebook_Shared          0
Income                   0
Education_Score          0
Age                      0
First_Interaction        0
City_Type                0
Employer_Category        0
Camp_Start_Date          0
Camp_End_Date            0
Category1                0
Category2                0
Category3                0
Outcome              35249
dtype: int64

In [122]:
len(train)

74944

In [124]:
data.reset_index(drop=True, inplace=True)

In [136]:
data[data.Patient_ID==data.Patient_ID.unique()[0]].Registration_Date.sort_values()

42945   2004-12-06
12210   2004-12-18
37977   2004-12-30
621     2005-03-15
20757   2005-05-01
21276   2005-05-01
25324   2005-05-08
24044   2005-08-11
0       2005-09-10
1963    2005-11-26
81898   2006-04-18
Name: Registration_Date, dtype: datetime64[ns]

In [160]:
data['camp_duration'] = data.apply(lambda x: x.Camp_End_Date.toordinal()-x.Camp_Start_Date.toordinal(), axis=1)
data['camp_start_registration'] = data.apply(lambda x: x.Registration_Date.toordinal()-x.Camp_Start_Date.toordinal(), axis=1)
data['camp_registration_end'] = data.apply(lambda x: x.Camp_End_Date.toordinal()-x.Registration_Date.toordinal(), axis=1)
data['camp_start_first_interaction'] = data.apply(lambda x: x.First_Interaction.toordinal()-x.Camp_Start_Date.toordinal(), axis=1)
data['camp_registration_first_interaction'] = data.apply(lambda x: x.Registration_Date.toordinal()-x.First_Interaction.toordinal(), axis=1)

In [161]:
data

,Patient_ID,Health_Camp_ID,Registration_Date,Var1,Var2,Var3,Var4,Var5,Online_Follower,LinkedIn_Shared,...,Camp_End_Date,Category1,Category2,Category3,Outcome,camp_duration,camp_start_registration,camp_registration_end,camp_start_first_interaction,camp_registration_first_interaction
0,489652,6578,2005-09-10,4,0,0,0,2,0,0,...,2005-10-14,Third,G,2,1.0,59,25,34,-253,278
1,507246,6578,2005-08-18,45,5,0,0,7,0,0,...,2005-10-14,Third,G,2,0.0,59,2,57,-342,344
2,523729,6534,2006-04-29,0,0,0,0,0,0,0,...,2007-11-07,Second,A,2,1.0,751,194,557,-482,676
3,524931,6535,2004-02-07,0,0,0,0,0,0,0,...,2004-02-18,First,E,2,0.0,17,6,11,6,0
4,521364,6529,2006-02-28,15,1,0,0,7,0,0,...,2006-04-03,Second,A,2,1.0,4,-30,34,-1000,970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110188,507261,6525,2006-04-13,12,2,0,0,6,0,0,...,2006-04-17,First,C,2,NaN,9,5,4,-781,786
110189,513607,6551,2006-11-03,0,0,0,0,0,0,0,...,2006-11-18,Second,D,2,NaN,5,-10,15,-425,415
110190,498160,6584,2006-06-17,0,0,0,0,0,0,0,...,2006-08-09,Second,A,2,NaN,5,-48,53,-784,736
110191,502444,6567,2007-01-13,0,0,0,0,0,0,0,...,2007-02-04,Second,D,2,NaN,5,-17,22,-636,619


In [194]:
category1 = {}
for i,j in enumerate(['First', 'Second', 'Third']):
    category1[j] = i+1

category2 = {}
for i,j in enumerate(sorted(data.Category2.unique())):
    category2[j] = i+1    

In [197]:
data['Category1'] = data['Category1'].map(lambda x: category1[x])
data['Category2'] = data['Category2'].map(lambda x: category2[x])

In [184]:
data = data[['Patient_ID', 'Health_Camp_ID', 'Registration_Date', 'Var1', 'Var2',
       'Var3', 'Var4', 'Var5', 'Online_Follower', 'LinkedIn_Shared',
       'Twitter_Shared', 'Facebook_Shared', 'Income', 'Education_Score', 'Age',
       'First_Interaction', 'City_Type', 'Employer_Category',
       'Camp_Start_Date', 'Camp_End_Date', 'Category1', 'Category2',
       'Category3', 'camp_duration', 'camp_start_registration',
       'camp_registration_end', 'camp_start_first_interaction',
       'camp_registration_first_interaction', 'Outcome']]

In [187]:
data.columns

Index(['Patient_ID', 'Health_Camp_ID', 'Registration_Date', 'Var1', 'Var2',
       'Var3', 'Var4', 'Var5', 'Online_Follower', 'LinkedIn_Shared',
       'Twitter_Shared', 'Facebook_Shared', 'Income', 'Education_Score', 'Age',
       'First_Interaction', 'City_Type', 'Employer_Category',
       'Camp_Start_Date', 'Camp_End_Date', 'Category1', 'Category2',
       'Category3', 'camp_duration', 'camp_start_registration',
       'camp_registration_end', 'camp_start_first_interaction',
       'camp_registration_first_interaction', 'Outcome'],
      dtype='object')

In [215]:
keep_cols = ['Var1', 'Var2',
       'Var3', 'Var4', 'Var5', 'Online_Follower', 'LinkedIn_Shared',
       'Twitter_Shared', 'Facebook_Shared', 'Income', 'Education_Score', 'age_group', 'City_Type', 'Category1', 'Category2',
       'Category3', 'camp_duration', 'camp_start_registration',
       'camp_registration_end', 'camp_start_first_interaction',
       'camp_registration_first_interaction']

In [218]:
data[keep_cols]

,Var1,Var2,Var3,Var4,Var5,Online_Follower,LinkedIn_Shared,Twitter_Shared,Facebook_Shared,Income,...,age_group,City_Type,Category1,Category2,Category3,camp_duration,camp_start_registration,camp_registration_end,camp_start_first_interaction,camp_registration_first_interaction
0,4,0,0,0,2,0,0,0,0,7,...,1,J,3,7,2,59,25,34,-253,278
1,45,5,0,0,7,0,0,0,0,1,...,2,C,3,7,2,59,2,57,-342,344
2,0,0,0,0,0,0,0,0,0,7,...,1,J,2,1,2,751,194,557,-482,676
3,0,0,0,0,0,0,0,0,0,7,...,1,I,1,5,2,17,6,11,6,0
4,15,1,0,0,7,0,0,0,1,1,...,2,I,2,1,2,4,-30,34,-1000,970
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110188,12,2,0,0,6,0,0,1,1,1,...,1,J,1,3,2,9,5,4,-781,786
110189,0,0,0,0,0,0,0,0,0,7,...,1,G,2,4,2,5,-10,15,-425,415
110190,0,0,0,0,0,0,0,0,0,7,...,1,A,2,1,2,5,-48,53,-784,736
110191,0,0,0,0,0,0,0,0,0,7,...,1,J,2,4,2,5,-17,22,-636,619


In [222]:
data['City_Type'] = data.City_Type.map(data.groupby('City_Type').size()/len(data))

In [231]:
data.Category2.unique()

array([7, 1, 5, 6, 4, 2, 3])

In [237]:
train = data[data.Outcome.notnull()]
test = data[data.Outcome.isnull()]

In [239]:
train = train.sort_values(['Camp_Start_Date', 'Camp_End_Date']).reset_index(drop=True)

In [278]:
train_x, test_x, train_y, test_y = train_test_split(train[keep_cols], train.Outcome, test_size=0.3)

In [242]:
xgb = XGBClassifier()
xgb.fit(train_x, train_y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [279]:
lgb = LGBMClassifier()
lgb.fit(train_x, train_y)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [243]:
xgb.score(train_x, train_y)

0.8206252382767824

In [244]:
xgb.score(test_x, test_y)

0.8200053371286248

In [280]:
lgb.score(train_x, train_y)

0.834178421654594

In [281]:
lgb.score(test_x, test_y)

0.8235634228784914

In [271]:
sample['Outcome'] = lgb.predict_proba(test[keep_cols])[:,1]

In [272]:
sample.set_index('Patient_ID').to_csv('/Users/skylark/Desktop/healthcare/result.csv')

In [254]:
perm = PermutationImportance(xgb,random_state=100).fit(test_x, test_y)
eli5.show_weights(perm,feature_names=test_x.columns.tolist())

Weight,Feature
0.1683 ± 0.0067,Category1
0.0599 ± 0.0032,camp_start_registration
0.0025 ± 0.0005,camp_duration
0.0011 ± 0.0002,City_Type
0.0010 ± 0.0006,Category2
0.0008 ± 0.0011,Income
0.0007 ± 0.0005,Var1
0.0006 ± 0.0006,Education_Score
0.0005 ± 0.0002,camp_start_first_interaction
0.0005 ± 0.0005,camp_registration_first_interaction


### SFS

In [256]:
len(train_x.columns)

21

In [258]:
sfs = SFS(estimator=xgb, k_features=21, verbose=2, scoring='accuracy')
sfs.fit(train[keep_cols], train.Outcome)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    7.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  21 out of  21 | elapsed:  2.5min finished

[2020-08-27 20:49:34] Features: 1/21 -- score: 0.7399648777729574[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   10.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:  3.4min finished

[2020-08-27 20:53:01] Features: 2/21 -- score: 0.7700412626146703[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   12.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed:  3.6min finished

[2020-08-27 20:56:38] Features: 3/21 -- score: 0.7716825124572364[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  

SequentialFeatureSelector(clone_estimator=True, cv=5,
                          estimator=XGBClassifier(base_score=0.5,
                                                  booster='gbtree',
                                                  colsample_bylevel=1,
                                                  colsample_bynode=1,
                                                  colsample_bytree=1, gamma=0,
                                                  learning_rate=0.1,
                                                  max_delta_step=0, max_depth=3,
                                                  min_child_weight=1,
                                                  missing=None,
                                                  n_estimators=100, n_jobs=1,
                                                  nthread=None,
                                                  objective='binary:logistic',
                                                  random_state=0, reg_alpha=0,
        

In [266]:
best_feat = list(sfs.subsets_[11]['feature_names'])

In [282]:
train_x, test_x, train_y, test_y = train_test_split(train[best_feat], train.Outcome, test_size=0.3)

In [283]:
lgb = LGBMClassifier()
lgb.fit(train_x, train_y)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [284]:
lgb.score(train_x, train_y)

0.8245711017918415

In [285]:
lgb.score(test_x, test_y)

0.8240081835972247

In [287]:
sample['Outcome'] = lgb.predict_proba(test[best_feat])[:,1]

In [288]:
sample.set_index('Patient_ID').to_csv('/Users/skylark/Desktop/healthcare/result.csv')